In [ ]:
import copy
import json

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import requests
import seaborn as sns
import spectrum_utils.spectrum as sus
import spectrum_utils.plot as sup
from matplotlib.lines import Line2D

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='serif')
sns.set_palette(['#9e0059', '#6da7de', '#ee266d', '#dee000', '#eb861e'])
sns.set_context('paper')

In [ ]:
usis = [
    'mzspec:GNPS:GNPS-LIBRARY:accession:CCMSLIB00003135669',
    'mzspec:MSV000085561:011c.mzML:scan:2864',
    'mzspec:MSV000085561:010c.mzML:scan:2829',
    'mzspec:MSV000085561:018b.mzML:scan:2609',
    'mzspec:MSV000082650:M031_48.mzML:scan:1501'
]
compounds = [
    'Hexanoylcarnitine',
    'Hexanoylcarnitine - 2.016 Da',
    'Hexanoylcarnitine + 5.955 Da\n(Benzoylcarnitine)',
    'Hexanoylcarnitine + 15.995 Da\n(Hydroxyhexanoylcarnitine)',
    'Hexanoylcarnitine + 114.068 Da\n(Dodecanedioylcarnitine)',
]

In [ ]:
spectra = []
for usi in usis:
    spectrum_dict = json.loads(requests.get(
        f'https://metabolomics-usi.ucsd.edu/json/?usi1={usi}',
        timeout=None).text)
    mz, intensity = zip(*spectrum_dict['peaks'])
    spectra.append(sus.MsmsSpectrum(
        usi, spectrum_dict['precursor_mz'], 0, mz, intensity))

In [ ]:
sup.colors['C4H5O2'] = '#9e0059'
sup.colors['C7H14NO2'] = '#6da7de'
sup.colors['TMAI'] = '#63c5b5'

fig, axes = plt.subplots(len(spectra), 1, sharex=True, sharey=True,
                         figsize=(4, len(spectra) * 1.5))
axes = axes.ravel()

for i, (spec, ax) in enumerate(zip(spectra, axes)):
    spec = copy.copy(spec)
    spec.set_mz_range(min_mz=0, max_mz=400).remove_precursor_peak(1, 'Da')
    # Peak annotations.
    try:
        spec.annotate_mz_fragment(
            85.0284, 1, 0.1, 'Da', text='C$_4$H$_5$O$_2$')
    except ValueError:
        pass
    try:
        spec.annotate_mz_fragment(60.0808, 1, 0.1, 'Da', text='TMAI')
    except ValueError:
        pass
    try:
        spec.annotate_mz_fragment(
            144.1019, 1, 0.1, 'Da', text='C$_7$H$_{14}$NO$_2$')
    except ValueError:
        pass
    # Loss annotations.
    # TMA (trimethylamine) loss: 59.0735
    ax.axvspan(spec.precursor_mz - 59.0735, spec.precursor_mz,
               color='#d82222', alpha=0.4, lw=0)
    # C7H14NO2 (carnitine - H2O) loss: 144.1019
    ax.axvspan(spec.precursor_mz - 144.1019, spec.precursor_mz - 59.0735,
               color='#eb861e', alpha=0.4, lw=0)
    # Carnitine loss: 161.1052
    ax.axvspan(spec.precursor_mz - 161.1052, spec.precursor_mz - 144.1019,
               color='#dee000', alpha=0.4, lw=0)
    
    for annotation in spec.annotation:
        if annotation is not None:
            if annotation.calc_mz == 60.0808:
                annotation.ion_type = 'TMAI'
            elif annotation.calc_mz == 85.0284:
                annotation.ion_type = 'C4H5O2'
            elif annotation.calc_mz == 144.1019:
                annotation.ion_type = 'C7H14NO2'
    
    sup.spectrum(spec, grid=False, ax=ax)
    ax.set_xlim(0, 450)
    ax.set_ylim(0, 1)

for ax, compound, spec in zip(axes, compounds, spectra):
    color = 'darkgray' if compound == 'Hexanoylcarnitine' else 'black'
    ax.set_title(f'{compound}\n$m$/$z$ = {spec.precursor_mz:.3f}',
                 loc='right', color=color, math_fontfamily='dejavuserif')
    
handles = [
    Line2D([0], [0], color='#dee000', lw=2, solid_capstyle='butt', alpha=0.4),
    Line2D([0], [0], color='#eb861e', lw=2, solid_capstyle='butt', alpha=0.4),
    Line2D([0], [0], color='#d82222', lw=2, solid_capstyle='butt', alpha=0.4),
]
fig.legend(handles, ['Carnitine loss', 'C$_7$H$_{14}$NO$_2$ loss', 'TMA loss'],
           loc='lower center', bbox_to_anchor=(0.5, 0.98), ncol=3,
           handlelength=1.2)

for ax in axes[:-1]:
    ax.set_xlabel('')
    
for ax in axes:
    sns.despine(ax=ax)
    
plt.tight_layout()
    
plt.savefig('acylcarnitine_suspects.png', bbox_inches='tight', dpi=300)
plt.show()
plt.close()